In [ ]:
import pandas as pd

data = pd.read_csv('c:/Users/dani/PycharmProjects/WOW/CSV/TCUSTCOUPONITEMSSPENT.csv', index_col=None)
data = data.loc[(data.ACCOUNT_CODE == 10869649) & (data.DEAL_CODE == 20) & (data.COUPON_CODE == 2158)]
data.to_csv('User10869649.csv')
print(data)

In [ ]:
data = pd.read_csv('User10869649.csv')
d = "2013-W26"
import datetime
data['datestr'] = data[['YEARIND', 'WEEKIND']].apply(lambda x: '20{}-W{}-0'.format(x.YEARIND, x.WEEKIND), axis=1)
data['date'] = pd.to_datetime(data.datestr , format='%Y-W%W-%w')
print(data)

In [ ]:
import numpy as np
data = pd.read_csv('User10869649.csv', index_col=0).reset_index(drop=True)
data2 = pd.read_csv('User1.csv', index_col=0)
data2['SPENT'] = 0
#data = data.combine_first(data2).fillna(0) #method='ffill'
data = pd.merge(left=data, right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
data['SPENT2'] = data.SPENT_x + data.SPENT_y
data.SPENT2 = data.SPENT2.fillna(0)
data = data.fillna(method='ffill')
print(data)
data.to_csv('User2.csv')

In [21]:
def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen

def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals

def triple_exponential_smoothing(series, slen=4, alpha=0.716, beta=0.029, gamma=0.993, n_preds=1):
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result

In [24]:
import numpy as np
import pandas as pd
data2 = pd.read_csv('User1.csv', index_col=0)
data2['SPENT'] = 0
data = pd.read_csv('User10869649.csv', index_col=0).reset_index(drop=True)

def f(group):
   df = pd.merge(left=group.reset_index(), right=data2, on=['WEEKIND', 'YEARIND'], how='outer')
   df['SPENT2'] = df.SPENT_x + df.SPENT_y
   df.SPENT2 = df.SPENT2.fillna(0)
   df['DATE'] = 100*df.YEARIND + df.WEEKIND
   df.sort_values(['DATE'], inplace=True)
   df = df.fillna(method='ffill')
   df = df.dropna()
   df = df.tail(10)
   df = df.reset_index()
   print(df)
   hws = triple_exponential_smoothing(df.SPENT2)
   #df.to_csv('User.csv')
   return hws[-1] #group.SPENT.min()

grouped = data.groupby(['DEAL_CODE', 'ACCOUNT_CODE', 'COUPON_CODE']).apply(f)
print(grouped)

   level_0  index  DEAL_CODE  ACCOUNT_CODE  COUPON_CODE  SPENT_x  WEEKIND  \
0       21   53.0       20.0         200.0        100.0     90.0       22   
1       24   56.0       20.0         200.0        100.0    120.0       23   
2       31   63.0       20.0         200.0        100.0    120.0       24   
3        8   40.0       20.0         200.0        100.0    240.0       25   
4        7   39.0       20.0         200.0        100.0    360.0       26   
5       14   46.0       20.0         200.0        100.0    120.0       27   
6       25   57.0       20.0         200.0        100.0    120.0       28   
7       52   57.0       20.0         200.0        100.0    120.0       29   
8        2   34.0       20.0         200.0        100.0    120.0       30   
9       12   44.0       20.0         200.0        100.0    120.0       31   

   YEARIND  SPENT_y  SPENT2  DATE  
0       17        0    90.0  1722  
1       17        0   120.0  1723  
2       17        0   120.0  1724  
3       